In [21]:
%load_ext bigdata
%hive_start
%timeout 300

The bigdata extension is already loaded. To reload it, use:
  %reload_ext bigdata


In [22]:
%%hive
DROP TABLE IF EXISTS tbl0;
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
--
DROP TABLE IF EXISTS tbl1;
CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1;

DROP TABLE IF EXISTS tbl0;
OK
Time taken: 21.12 seconds
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 1.469 seconds
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
Loading data to table default.tbl0
OK
Time taken: 2.341 seconds
--
DROP TABLE IF EXISTS tbl1;
OK
Time taken: 0.235 seconds
CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 0.124 seconds
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1;
Loading data to table default.tbl1
OK
Time taken: 1.414 seconds


In [24]:
%%hive 
DROP TABLE IF EXISTS dataTable; 
CREATE TABLE dataTable (id INT, letraM ARRAY<CHAR(1)>);
INSERT OVERWRITE TABLE dataTable 
SELECT c1, collect_list(upper(exploded)) FROM tbl0 LATERAL VIEW explode(c5) exploded_table AS exploded GROUP BY c1;

DROP TABLE IF EXISTS dataTable; 
OK
Time taken: 0.331 seconds
CREATE TABLE dataTable (id INT, letraM ARRAY<CHAR(1)>);
OK
Time taken: 0.126 seconds
INSERT OVERWRITE TABLE dataTable 
) exploded_table AS exploded GROUP BY c1;FROM tbl0 LATERAL VIEW explode(c5 
Query ID = root_20200307012217_0c9bf268-45e9-4659-9e82-e75509fe3a63
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1583536974433_0040, Tracking URL = http://4775b5941e40:8088/proxy/application_1583536974433_0040/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1583536974433_0040
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
20

In [38]:
%%hive 
SELECT letraM FROM dataTable;

SELECT letraM FROM dataTable;
OK
["A","E","C","D"]
["A","C"]
["B","D"]
["A","C","E","B","D"]
["D","E","C"]
["A","D"]
["A","E","D"]
["D","A","C","E"]
["B","A","C","E"]
["C","D","E","A","B"]
Time taken: 24.283 seconds, Fetched: 10 row(s)


In [26]:
%hive_quit

In [47]:
## Copia Archivos a sistema HDFS
##
## Se usan un directorio temporal en el HDFS. La siguiente
## instrucción muestra el contenido del dicho directorio
##
!hdfs dfs -ls /tmp

Found 4 items
drwxrwx---   - root supergroup          0 2020-03-06 23:24 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-03-06 23:27 /tmp/hive
drwxrwxrwx   - root supergroup          0 2020-03-07 01:27 /tmp/output
drwxr-xr-x   - root supergroup          0 2020-03-07 01:23 /tmp/wordcount


In [48]:
## Crea la carpeta wordcount en el hdfs
##
!hdfs dfs -mkdir /tmp/wordcount

mkdir: `/tmp/wordcount': File exists


In [49]:
## Verifica la creación de la carpeta
##
!hdfs dfs -ls /tmp/

Found 4 items
drwxrwx---   - root supergroup          0 2020-03-06 23:24 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-03-06 23:27 /tmp/hive
drwxrwxrwx   - root supergroup          0 2020-03-07 01:27 /tmp/output
drwxr-xr-x   - root supergroup          0 2020-03-07 01:23 /tmp/wordcount


In [50]:
## Copia los archvios del directorio local wordcount/
## al directorio /tmp/wordcount/ en el hdfs
##
!hdfs dfs -copyFromLocal 'data.tsv'  /tmp/wordcount/

copyFromLocal: `data.tsv': No such file or directory


In [51]:
##
## Verifica que los archivos esten copiados
## en el hdfs
##
!hdfs dfs -ls /tmp/wordcount

In [32]:
####################################################################################################################
## Generación del script y ajuste del código

In [52]:
%%writefile wordcount.hql

DROP TABLE IF EXISTS tbl0;
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
--
DROP TABLE IF EXISTS tbl1;
CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1;


DROP TABLE IF EXISTS dataTable; 
CREATE TABLE dataTable (id INT, letraM ARRAY<CHAR(1)>);
INSERT OVERWRITE TABLE dataTable 
SELECT c1, collect_list(upper(exploded)) FROM tbl0 LATERAL VIEW explode(c5) exploded_table AS exploded GROUP BY c1;

INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'

SELECT letraM FROM dataTable;

Overwriting wordcount.hql


In [53]:
## Ejecucion
!hive -S -e 'source wordcount.hql'

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
FAILED: SemanticException org.apache.hadoop.hive.ql.metadata.HiveException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient


In [54]:
## Copia de los resultados a la maquina local (vagrant)
!hadoop fs -copyToLocal /tmp/output output
!ls output/*

output/000000_0


In [55]:
!cat output/000000_0 | head

AECD
AC
BD
ACEBD
DEC
AD
AED
DACE
BACE
CDEAB
